In [ ]:
import tensorflow as tf
from keras_preprocessing import image
from keras_preprocessing.image import ImageDataGenerator

import matplotlib.pyplot as plt

import os
import stat
import math
import random
import zipfile
import shutil

from tqdm import tqdm

In [ ]:
%matplotlib inline

In [ ]:
config = tf.compat.v1.ConfigProto()
config.gpu_options.allow_growth = True
config.gpu_options.per_process_gpu_memory_fraction = 0.4
session = tf.compat.v1.Session(config=config)
tf.compat.v1.keras.backend.set_session(session)

In [ ]:
def get_model_memory_usage(batch_size, model):
    import numpy as np
    try:
        from keras import backend as K
    except:
        from tensorflow.keras import backend as K

    shapes_mem_count = 0
    internal_model_mem_count = 0
    for l in model.layers:
        layer_type = l.__class__.__name__
        if layer_type == 'Model':
            internal_model_mem_count += get_model_memory_usage(batch_size, l)
        single_layer_mem = 1
        out_shape = l.output_shape
        if type(out_shape) is list:
            out_shape = out_shape[0]
        for s in out_shape:
            if s is None:
                continue
            single_layer_mem *= s
        shapes_mem_count += single_layer_mem

    trainable_count = np.sum([K.count_params(p) for p in model.trainable_weights])
    non_trainable_count = np.sum([K.count_params(p) for p in model.non_trainable_weights])

    number_size = 4.0
    if K.floatx() == 'float16':
        number_size = 2.0
    if K.floatx() == 'float64':
        number_size = 8.0

    total_memory = number_size * (batch_size * shapes_mem_count + trainable_count + non_trainable_count)
    gbytes = np.round(total_memory / (1024.0 ** 3), 3) + internal_model_mem_count
    return gbytes

In [ ]:
get_model_memory_usage(BATCH_SIZE, model)

## !Extracting data!

In [ ]:
LOCAL_ZIP = './Data/data.zip'

In [ ]:
zip_ref = zipfile.ZipFile(LOCAL_ZIP, 'r')

tqdm(zip_ref.extractall('./Data/'))
zip_ref.close()


## Code Start

In [ ]:
CLEAN_DIR = './Data/Clean/'
DIRTY_DIR = './Data/Dirty/'
TRAINING_DIR = './Data/Training/'
TESTING_DIR = './Data/Testing/'
DEV_DIR = './Data/Dev/'

SPLIT_DIST = 0.1

batik_classes = os.listdir(CLEAN_DIR)

## Changing file permission

In [ ]:
def change_perm(target):
    for batik_class in batik_classes:
        dir = os.path.join(target, batik_class)
        dir_content = os.listdir(dir)

        for image in tqdm(dir_content, ascii = False, desc = batik_class):
            dst_dir = os.path.join(dir, image)
            os.chmod(dst_dir, stat.S_IRWXU)

In [ ]:
#change_perm(CLEAN_DIR)
change_perm(DIRTY_DIR)

----------------------------------------------
# !RUN WITH CAUTION!
## Data Splitting
 Making the DIR necessary for storing images and spreading it from CLEAN_DIR to apropriate dir

In [ ]:
def move_image(target):
    for batik_class in batik_classes:
        dir = os.path.join(target, batik_class)
        dir_content = os.listdir(dir)

        # Calculating how much image for splitting
        split_dist = math.ceil(len(dir_content) * SPLIT_DIST)

        # Picking random images
        testing_images = random.choices(dir_content, k = split_dist)

        # Moving things around
        for image in tqdm(dir_content, desc = batik_class):
            # If image is picked for testing
            if image in testing_images:
                src_dir = os.path.join(dir, image)
                dst_dir = os.path.join(TESTING_DIR, batik_class)

                shutil.copy(src_dir, dst_dir)
            else:
                src_dir = os.path.join(dir, image)
                dst_dir = os.path.join(TRAINING_DIR, batik_class)

                shutil.copy(src_dir, dst_dir)

In [ ]:
# MKDIR data subdir
try:
    os.mkdir(TRAINING_DIR)
    os.mkdir(TESTING_DIR)
    os.mkdir(DEV_DIR)
except:
    print('Data subdir folder already exist')

# MKDIR working subdir
for batik_class in batik_classes:
    try:
        dir = os.path.join(TRAINING_DIR, batik_class)
        os.mkdir(dir)

        dir = os.path.join(TESTING_DIR, batik_class)
        os.mkdir(dir)

        dir = os.path.join(DEV_DIR, batik_class)
        os.mkdir(dir)
    except:
        print('Working subdir folder already exist')
        break

move_image(CLEAN_DIR)
move_image(DIRTY_DIR)

## Purge images from working dir

In [ ]:
def purge_working_image(target):
    for batik_class in tqdm(batik_classes, desc = target):
        dir = os.path.join(target, batik_class)
        
        #print('Deleting ' + dir)
        shutil.rmtree(dir, ignore_errors = True)

In [ ]:
purge_working_image(TRAINING_DIR)
purge_working_image(TESTING_DIR)
purge_working_image(DEV_DIR)

----------------------------------------------

## Counting total images

In [ ]:
def count_batik(target):
    try:
        batik_counter = {}

        for batik_class in batik_classes:
            dir = os.path.join(target, batik_class)
            dir_content = os.listdir(dir)

            batik_counter[batik_class] = 0
            for image in dir_content:
                batik_counter[batik_class] += 1
    except:
        print('Directory Empty!')
        print()
    else:
        for i in batik_counter:
            print(i + ' = ' + str(batik_counter[i]))

        print()
        print('Total images: ' + str(sum(batik_counter.values())))

In [ ]:
print('CLEAN IMAGES')
print('-------------------------------------')

count_batik(CLEAN_DIR)

print('DIRTY IMAGES')
print('-------------------------------------')

count_batik(DIRTY_DIR)

In [ ]:
print('TRAINING IMAGES')
print('-------------------------------------')

count_batik(TRAINING_DIR)

print('TESTING IMAGES')
print('-------------------------------------')

count_batik(TESTING_DIR)

print('DEV IMAGES')
print('-------------------------------------')

count_batik(DEV_DIR)

## Data Augmentation

In [ ]:
IMAGE_SHAPE = [150, 150]
BATCH_SIZE = 256

In [ ]:
training_datagen = ImageDataGenerator(
    # Might need further augmentation
    rescale = 1/255.0,
    rotation_range = 40,
    width_shift_range = 0.2,
    height_shift_range = 0.2,
    shear_range = 0.2,
    zoom_range = 0.2,
    horizontal_flip = True
)

testing_datagen = ImageDataGenerator(
    rescale = 1/255.0,
)

training_generator = training_datagen.flow_from_directory(
    TRAINING_DIR,
    target_size=IMAGE_SHAPE,
    class_mode='categorical',
    batch_size=BATCH_SIZE
)

testing_generator = testing_datagen.flow_from_directory(
    TESTING_DIR,
    target_size=IMAGE_SHAPE,
    class_mode='categorical',
    batch_size=BATCH_SIZE
)

## Model

In [ ]:
mobilenetV3_model = tf.keras.applications.MobileNetV3Large(
    include_top=False,
    input_shape=IMAGE_SHAPE + [3]
)

efn_model = tf.keras.applications.EfficientNetB0(
    include_top=False,
    input_shape=IMAGE_SHAPE + [3]
)

nasnetmobile_model = tf.keras.applications.NASNetMobile(
    include_top=False,
    weights=None,
    input_shape=IMAGE_SHAPE + [3]
)

In [ ]:
for layer in mobilenetV3_model.layers:
    layer.trainable = False

In [ ]:
mobilenetV3_model.summary()

In [ ]:
global_average_layer = tf.keras.layers.GlobalAveragePooling2D()
output_layer = tf.keras.layers.Dense(20, activation='softmax')
model = tf.keras.Sequential([
    mobilenetV3_model,
    global_average_layer,
    output_layer
])

#Vanilla model

model = tf.keras.models.Sequential([
    tf.keras.layers.Conv2D(64, (3, 3), activation='relu', input_shape=(IMAGE_SHAPE + [3])),
    tf.keras.layers.MaxPool2D(2, 2),
    tf.keras.layers.Conv2D(64, (3, 3), activation='relu'),
    tf.keras.layers.MaxPool2D(2, 2),
    tf.keras.layers.Flatten(),
    tf.keras.layers.Dense(256, activation = 'relu'),
    #20 Batik class, might add more later?
    tf.keras.layers.Dense(20, activation = 'softmax')
])

model.summary()


## Compiling

Using time-based decay learning rate

In [ ]:
LEARNING_RATE = 0.01
EPOCHS = 550

decay = LEARNING_RATE / EPOCHS

ACCURACY_THRESHOLD = 0.8

In [ ]:
class myCallback(tf.keras.callbacks.Callback): 
    def on_epoch_end(self, epoch, logs={}):
        if(logs.get('accuracy') > ACCURACY_THRESHOLD):   
            print("\nReached %2.2f%% accuracy, so stopping training!!" %(ACCURACY_THRESHOLD*100))
            self.model.stop_training = True

CHECKPOINT_DIR = 'Checkpoint/'

checkpoint = tf.keras.callbacks.ModelCheckpoint(
    CHECKPOINT_DIR,
    monitor='accuracy',
    save_best_only=False,
    save_weights_only=True,
    verbose=1,
    save_freq=BATCH_SIZE*5
)

def lr_time_based_decay(epoch, lr):
    result = lr * 1 / (1 + decay * epoch)
    print('lr = ' + str(result))
    return result

lr_scheduler = tf.keras.callbacks.LearningRateScheduler(
    lr_time_based_decay
)

early_stop = tf.keras.callbacks.EarlyStopping(
    monitor='val_accuracy', 
    patience=10,
    min_delta=0.001, 
)

In [ ]:
callback = myCallback()

In [ ]:
model.compile(
    loss=tf.keras.losses.CategoricalCrossentropy(),
    optimizer=tf.keras.optimizers.Adam(learning_rate=LEARNING_RATE),
    metrics=['accuracy']
)

## Training

In [ ]:
history = model.fit(
    training_generator,
    validation_data=testing_generator,
    epochs=EPOCHS,
    callbacks=[callback, checkpoint, lr_scheduler, early_stop]
)

In [ ]:
model.load_weights(CHECKPOINT_DIR)

## Graphing

In [ ]:
acc = history.history['accuracy']
val_acc = history.history['val_accuracy']
loss = history.history['loss']
val_loss = history.history['val_loss']

epochs = range(len(acc))

fig, axs = plt.subplots(2)
axs[0].plot(epochs, acc, 'b', label='Training accuracy')
axs[0].plot(epochs, val_acc, 'y', label='Validation accuracy')
axs[0].legend(loc=0)

axs[1].plot(epochs, loss, 'b', label='Training loss')
axs[1].plot(epochs, val_loss, 'y', label='Validation loss')
axs[1].set_xlabel('Epoch')
axs[1].set_ylabel('Value')
axs[1].legend(loc=0)